#### source: https://github.com/adventuresinML/adventures-in-ml-code/blob/master/gensim_word2vec.py

In [ ]:
# import sys
# !{sys.executable} -m pip install numpy pandas matplotlib sklearn seaborn
# !{sys.executable} -m pip install --upgrade gensim
# !{sys.executable} -m pip install nltk
# import nltk
# nltk.download('punkt')

In [25]:
import logging
import collections
import os
import zipfile
import json
import numpy as np
import gensim
import tensorflow as tf

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing  import sequence
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.layers import Input, Dense, Embedding, Dot, Reshape, GlobalAveragePooling1D
from tensorflow.keras.models import Model

from urllib import request
from nltk.tokenize import word_tokenize

def get_data(filename="synonym.json"):
    documents = []
    with open(filename) as f:
        data = json.load(f)
        for item in data:
            documents.append(item.split(' '))
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]
    return documents

In [28]:
def evaluate(model, documents, step):
    percentiles = np.zeros(step)
    scores = np.zeros(step)
    for epoch in range(step):
        doc_id = np.random.randint(model.docvecs.count)
        print('\nfor doc-id {} -> {}'.format(doc_id, documents[doc_id]))
        inferred_vector  = model.infer_vector(documents[doc_id][0])
        similars = model.docvecs.most_similar(positive=[inferred_vector], topn=10)
        for sim in similars:
            print()
        for idx, item in enumerate(similars):
            if item[0] == documents[doc_id][1][0]:
                percentiles[epoch] = ((idx + 1) / len(similars)) * 100
                scores[epoch] = item[1]
                break
    return np.mean(percentiles), np.mean(scores)

max_epochs = 100
def train(model_name="alodokter-doc2vec.model"):
    documents = get_data()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = Doc2Vec(vector_size=10, min_count=2, epochs=1)
    model.build_vocab(documents)
    for epoch in range(max_epochs):
        print('\ntraining epoch {:d} ...'.format(epoch))
        model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
        mean_percentiles, accuracy = evaluate(model, documents, 100)
        print('mean percentiles: {:.2f}, accuracy: {:.2f}'.format(mean_percentiles, accuracy))
        
        model.alpha -= 0.002  # decrease the learning rate
        model.min_alpha = model.alpha  # fix the learning rate, no decay
        
#         test_data = word_tokenize("bab".lower())
#         inferred_vector = model.infer_vector(test_data)
#         similarities = model.docvecs.most_similar([inferred_vector], topn=1)
#         print("Similar Text: {} with Score: {}".format(documents[sims[0]][0], sims[1]))
    
    model.save(model_name)
    return model, documents

In [29]:
model, documents = train()
# model= Doc2Vec.load("alodokter-doc2vec.model")
# test_data = word_tokenize("bab".lower())
# inferred_vector = model.infer_vector(test_data)
# similarities = model.docvecs.most_similar([inferred_vector], topn=30)
# similarities
# # for sims in similarities:
# #     print("Similar Text: {} with Score: {}".format(documents[sims[0]][0], sims[1]))

2020-02-19 16:12:23,846 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-02-19 16:12:23,847 : INFO : collecting all words and their counts
2020-02-19 16:12:23,847 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-02-19 16:12:23,864 : INFO : PROGRESS: at example #10000, processed 25193 words (1520409/s), 3637 word types, 10000 tags
2020-02-19 16:12:23,883 : INFO : PROGRESS: at example #20000, processed 58270 words (1816052/s), 4826 word types, 20000 tags
2020-02-19 16:12:23,900 : INFO : PROGRESS: at example #30000, processed 82180 words (1441408/s), 7377 word types, 30000 tags
2020-02-19 16:12:23,920 : INFO : PROGRESS: at example #40000, processed 106394 words (1271767/s), 10759 word types, 40000 tags
2020-02-19 16:12:23,940 : INFO : PROGRESS: at example #50000, processed 132171 words (1315207/s), 12716 word types, 50000 tags
2020-02-19 16:12:23,962 : INFO : PROGRESS: at example #60000, processed 165988 words (165


training epoch 0 ...


2020-02-19 16:12:44,865 : INFO : EPOCH 1 - PROGRESS: at 17.60% examples, 54195 words/s, in_qsize 6, out_qsize 0
2020-02-19 16:12:46,160 : INFO : EPOCH 1 - PROGRESS: at 38.47% examples, 57227 words/s, in_qsize 5, out_qsize 0
2020-02-19 16:12:47,405 : INFO : EPOCH 1 - PROGRESS: at 61.90% examples, 64908 words/s, in_qsize 6, out_qsize 0
2020-02-19 16:12:48,512 : INFO : EPOCH 1 - PROGRESS: at 82.64% examples, 70019 words/s, in_qsize 6, out_qsize 0
2020-02-19 16:12:49,270 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-19 16:12:49,324 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-19 16:12:49,336 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-19 16:12:49,337 : INFO : EPOCH - 1 : training on 377314 raw words (429163 effective words) took 5.8s, 74303 effective words/s
2020-02-19 16:12:49,337 : INFO : training on a 377314 raw words (429163 effective words) took 5.8s, 74107 effective words/s
2020-02-19 16:12:49,3


for doc-id 32255 -> TaggedDocument(['krim', 'skabies'], [32255])
[(51800, 0.9646957516670227), (9831, 0.9615930914878845), (91729, 0.9592198729515076), (37675, 0.9517938494682312), (47006, 0.9481594562530518), (41236, 0.9443621635437012), (64131, 0.9436671733856201), (81629, 0.9376924633979797), (62709, 0.933881938457489), (65379, 0.9330042600631714)]

for doc-id 86724 -> TaggedDocument(['pori', 'pori', 'saya', 'terlihat', 'besar'], [86724])
[(14962, 0.9483673572540283), (109716, 0.9394255876541138), (18066, 0.9326097965240479), (81502, 0.9324712753295898), (76847, 0.9291368722915649), (41117, 0.9287288784980774), (90333, 0.9226935505867004), (15851, 0.9202475547790527), (82181, 0.9191737771034241), (33444, 0.9102692604064941)]

for doc-id 55485 -> TaggedDocument(['tumbuh', 'besar'], [55485])
[(29796, 0.9736717343330383), (51096, 0.9516332149505615), (45497, 0.9215255975723267), (5991, 0.9120280146598816), (12969, 0.9008965492248535), (52285, 0.8996639847755432), (49404, 0.89460563659

2020-02-19 16:12:49,566 : WARNING : Effective 'alpha' higher than previous training cycles
2020-02-19 16:12:49,567 : INFO : training model with 3 workers on 8548 vocabulary and 10 features, using sg=0 hs=0 sample=0.001 negative=5 window=5


...


for doc-id 17903 -> TaggedDocument(['urat', 'halus'], [17903])
[(90386, 0.9356622695922852), (7551, 0.9356464147567749), (72717, 0.9339258670806885), (15871, 0.9282909631729126), (78796, 0.9223120212554932), (6321, 0.9210814237594604), (83125, 0.9199354648590088), (51237, 0.9195102453231812), (91849, 0.9136338233947754), (15776, 0.911078691482544)]

for doc-id 55854 -> TaggedDocument(['nafas', 'saya', 'serasa', 'seperti', 'tertahan'], [55854])
[(71202, 0.9461773633956909), (100169, 0.9321778416633606), (90878, 0.9296311140060425), (96197, 0.9270145893096924), (33723, 0.9231677055358887), (68220, 0.9204826354980469), (110027, 0.9186018705368042), (84921, 0.9179864525794983), (9692, 0.9125550985336304), (70365, 0.9087598323822021)]

for doc-id 76218 -> TaggedDocument(['pendengaran', 'saya', 'sedikit', 'berkurang'], [76218])
[(94355, 0.9631910920143127), (71573, 0.9592152833938599), (20292, 0.9405961632728577), (7395, 0.9390233159065247), (85813, 0.9376150965690613), (59474, 0.9361335635

2020-02-19 16:17:11,143 : INFO : training model with 3 workers on 8548 vocabulary and 10 features, using sg=0 hs=0 sample=0.001 negative=5 window=5


[(111099, 0.9701228737831116), (37966, 0.9425927996635437), (107115, 0.9405866861343384), (67285, 0.9401171207427979), (75283, 0.9288735389709473), (82855, 0.9283545613288879), (18398, 0.9267467260360718), (15042, 0.9242100715637207), (62515, 0.9238677024841309), (62979, 0.9207228422164917)]

for doc-id 82777 -> TaggedDocument(['kurang', 'manjur', 'rasa', 'pahit'], [82777])
[(9709, 0.9639854431152344), (47843, 0.9509308338165283), (462, 0.9492520093917847), (97041, 0.9417341947555542), (7309, 0.9363024234771729), (53457, 0.9319707155227661), (56798, 0.9305365085601807), (40903, 0.9284743666648865), (32611, 0.9268354773521423), (2211, 0.9260916709899902)]

for doc-id 25742 -> TaggedDocument(['alergi', 'menthol'], [25742])
[(23692, 0.961984395980835), (9391, 0.9443018436431885), (22384, 0.9428762197494507), (68825, 0.9383447170257568), (37774, 0.9356783032417297), (4358, 0.9294617176055908), (50127, 0.9217542409896851), (45360, 0.906837522983551), (95468, 0.9049517512321472), (35860, 0.9

2020-02-19 16:17:12,197 : INFO : EPOCH 1 - PROGRESS: at 14.34% examples, 54551 words/s, in_qsize 6, out_qsize 0
2020-02-19 16:17:13,311 : INFO : EPOCH 1 - PROGRESS: at 31.78% examples, 57150 words/s, in_qsize 5, out_qsize 0


KeyboardInterrupt: 